In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction/Output')
cur_date = "051023"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)

# reading in files
well_data = data.frame(read_excel("Input/Imputed_Well_Data_050823.xlsx")) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
#Uighur

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


randomForest 4.7-1.1

Type rfNews() to see n

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,41176,52,165,41,60.0,7.7,Ar,1.321158,ND
2,1024009,1803,15-177,42355,40,445,42,2.0,7.3,Ar,1.010168,ND
3,1054019,1647,14-212,42037,45,160,40,40.0,7.4,Ar,1.358221,ND
4,1057017,970,12-169,41204,42,440,57,1.5,8.0,Ar,35.000000,D
5,1060006,533,10-239,40546,48,120,42,25.0,7.1,Ar,1.879384,ND
6,1066006,1795,15-224,42353,60,280,32,10.0,8.2,Ar,53.000000,D


Using `casing_depth`, `well_depth`, `static_water_depth`, `pH`, and `flow_rate` to predict As concentration. RF models will be built to predict concentration that has binarized in 3 different ways:

1. Detect vs. Non-Detect
2. < or > 5ppb
3. < or > 10ppb

Starting by creating 2 additional variables for above and below 5 and 10ppb and calculating some summary statistics to determine if there are any signficiant differenes between the predictor variables for each outcome variable.

In [3]:
arsenic_data = well_data %>%
    # filtering for Arsenic only
    filter(Metal == "Ar") %>%
    mutate(Five_Threshold = relevel(factor(ifelse(Concentration > 5, 1, 0)), ref = "0")) %>%
    mutate(Ten_Threshold = relevel(factor(ifelse(Concentration > 10, 1, 0)), ref = "0")) %>%
    # making this col into a factor
    mutate(Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0")) 

head(arsenic_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration,Five_Threshold,Ten_Threshold
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<fct>
1,1006004,960,12-155,41176,52,165,41,60.0,7.7,Ar,1.321158,0,0,0
2,1024009,1803,15-177,42355,40,445,42,2.0,7.3,Ar,1.010168,0,0,0
3,1054019,1647,14-212,42037,45,160,40,40.0,7.4,Ar,1.358221,0,0,0
4,1057017,970,12-169,41204,42,440,57,1.5,8.0,Ar,35.000000,1,1,1
5,1060006,533,10-239,40546,48,120,42,25.0,7.1,Ar,1.879384,0,0,0
6,1066006,1795,15-224,42353,60,280,32,10.0,8.2,Ar,53.000000,1,1,1


In [4]:
# summary statistics df: detect_concentration
arsenic_data %>%
  tbl_summary(by = Detect_Concentration, missing = "no", 
  include = colnames(arsenic_data[c(5:9,12)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 731","**1**, N = 292",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Casing_Depth,"1,023",73 (33),56 (23),<0.001
Well_Depth,"1,023",306 (148),340 (137),<0.001
Static_Water_Depth,"1,023",35 (14),37 (16),0.067
Flow_Rate,"1,023",25 (34),14 (17),<0.001
pH,"1,023",7.41 (0.55),7.78 (0.39),<0.001


In [5]:
# summary statistics df: five_threshold
arsenic_data %>%
  tbl_summary(by = Five_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(5:9,13)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 750","**1**, N = 273",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Casing_Depth,"1,023",73 (33),55 (22),<0.001
Well_Depth,"1,023",306 (149),341 (136),<0.001
Static_Water_Depth,"1,023",35 (14),37 (16),0.030
Flow_Rate,"1,023",25 (33),14 (17),<0.001
pH,"1,023",7.42 (0.55),7.79 (0.39),<0.001


In [6]:
# summary statistics df: ten_threshold
arsenic_data %>%
  tbl_summary(by = Ten_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(5:9,14)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 833","**1**, N = 190",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Casing_Depth,"1,023",72 (33),53 (20),<0.001
Well_Depth,"1,023",311 (151),336 (122),0.036
Static_Water_Depth,"1,023",35 (14),38 (16),0.009
Flow_Rate,"1,023",24 (32),13 (16),<0.001
pH,"1,023",7.45 (0.55),7.80 (0.36),<0.001


Although the p values are very significant between the classes of each outcome, there is a high level of class imbalance which is likely to affect model performance.

In [7]:
# creating dfs for each outcome
arsenic_detect_df = arsenic_data[,c(5:9,12)]  
arsenic_five_df = arsenic_data[,c(5:9,13)]  
arsenic_ten_df = arsenic_data[,c(5:9,14)] 

head(arsenic_detect_df)

,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Detect_Concentration
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,52,165,41,60.0,7.7,0
2,40,445,42,2.0,7.3,0
3,45,160,40,40.0,7.4,0
4,42,440,57,1.5,8.0,1
5,48,120,42,25.0,7.1,0
6,60,280,32,10.0,8.2,1


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [8]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) # number of trees 
        p = dim(dataset)[2] - 1 # number of variables in dataset
        mtry_values = c(sqrt(p), p/2, p/3)

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [9]:
# calling fn
rf_values_arsenic_detect = rf_classification(arsenic_detect_df, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
rf_values_arsenic_five = rf_classification(arsenic_five_df, "Five_Threshold", 
                                             "pred_Five_Threshold")
rf_values_arsenic_ten = rf_classification(arsenic_ten_df, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [10]:
# viewing results
rf_confusion_matrix = data.frame(Model = rep(c("RF Classification"), times = 3), Kernel = NA,
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
            rbind(rf_values_arsenic_detect[[2]], rf_values_arsenic_five[[2]], rf_values_arsenic_ten[[2]]))

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_detect[[3]]
rf_values_arsenic_five[[3]]
rf_values_arsenic_ten[[3]]

Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,NA,Detect_Concentration,0.6668666,0.4554646,0.8782686,0.6001878,0.6668666
RF Classification,NA,Five_Threshold,0.6354545,0.3842424,0.8866667,0.5486266,0.6354545
RF Classification,NA,Ten_Threshold,0.6130723,0.2789474,0.9471972,0.5467380,0.6130723


Predictor,MeanDecreaseGini
<chr>,<dbl>
Casing_Depth,80.47507
pH,73.97898
Flow_Rate,59.44930
Static_Water_Depth,58.10952
Well_Depth,56.38139


Predictor,MeanDecreaseGini
<chr>,<dbl>
Casing_Depth,75.57686
pH,69.82306
Flow_Rate,57.81790
Static_Water_Depth,56.88735
Well_Depth,54.77027


Predictor,MeanDecreaseGini
<chr>,<dbl>
Casing_Depth,56.01037
pH,48.99548
Static_Water_Depth,47.75311
Flow_Rate,44.98069
Well_Depth,44.90544


RF didn't perform as well as I hoped, so let's try another model - SVM. 

# Support Vector Machine (SVM)
- predicts by projecting predictors onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [11]:
# model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        # now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        # choosing best model
        best_svm <- svr_tune$best.model

        # predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  # taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [12]:
# calling fn
# linear
linear_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "linear", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
linear_svm_values_arsenic_five = svm_classification(arsenic_five_df, "linear", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
linear_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "linear", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# radial
radial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "radial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
radial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "radial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
radial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "radial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# polynomial
polynomial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "polynomial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, 0.1)
polynomial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "polynomial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, 0.1)
polynomial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "polynomial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [13]:
# viewing results
svm_confusion_matrix = data.frame(Model = "SVM Classification",
        Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
        Kernel = rep(c(rep(c("linear"), times = 3), rep(c("radial"), times = 3), rep(c("polynomial"), times = 3))),
            rbind(linear_svm_values_arsenic_detect, linear_svm_values_arsenic_five, linear_svm_values_arsenic_ten, 
            radial_svm_values_arsenic_detect, radial_svm_values_arsenic_five, radial_svm_values_arsenic_ten, 
            polynomial_svm_values_arsenic_detect, polynomial_svm_values_arsenic_five, 
            polynomial_svm_values_arsenic_ten))

svm_confusion_matrix

Model,Outcome,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,Detect_Concentration,linear,0.4996693,0.003448276,0.9958904,NaN,0.4996693
SVM Classification,Five_Threshold,linear,0.5000000,0.000000000,1.0000000,NaN,0.5000000
SVM Classification,Ten_Threshold,linear,0.5000000,0.000000000,1.0000000,NaN,0.5000000
SVM Classification,Detect_Concentration,radial,0.5861678,0.268030392,0.9043053,NaN,0.5861678
SVM Classification,Five_Threshold,radial,0.5071515,0.043636364,0.9706667,NaN,0.5071515
SVM Classification,Ten_Threshold,radial,0.5000000,0.000000000,1.0000000,NaN,0.5000000
SVM Classification,Detect_Concentration,polynomial,0.4996740,0.003448276,0.9958997,NaN,0.4996740
SVM Classification,Five_Threshold,polynomial,0.5016700,0.007340067,0.9960000,NaN,0.5016700
SVM Classification,Ten_Threshold,polynomial,0.5000000,0.000000000,1.0000000,NaN,0.5000000


In [14]:
# combining in 1 df
final_df = full_join(rf_confusion_matrix, svm_confusion_matrix) %>%
    arrange(Outcome, -Balanced.Accuracy)

final_df

Joining, by = c("Model", "Kernel", "Outcome", "Balanced.Accuracy",
"Sensitivity", "Specificity", "PPV", "AUC")


Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,NA,Detect_Concentration,0.6668666,0.455464641,0.8782686,0.6001878,0.6668666
SVM Classification,radial,Detect_Concentration,0.5861678,0.268030392,0.9043053,NaN,0.5861678
SVM Classification,polynomial,Detect_Concentration,0.4996740,0.003448276,0.9958997,NaN,0.4996740
SVM Classification,linear,Detect_Concentration,0.4996693,0.003448276,0.9958904,NaN,0.4996693
RF Classification,NA,Five_Threshold,0.6354545,0.384242424,0.8866667,0.5486266,0.6354545
SVM Classification,radial,Five_Threshold,0.5071515,0.043636364,0.9706667,NaN,0.5071515
SVM Classification,polynomial,Five_Threshold,0.5016700,0.007340067,0.9960000,NaN,0.5016700
SVM Classification,linear,Five_Threshold,0.5000000,0.000000000,1.0000000,NaN,0.5000000
RF Classification,NA,Ten_Threshold,0.6130723,0.278947368,0.9471972,0.5467380,0.6130723
